In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [14]:
#GPU Checking
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

Using GPU: NVIDIA GeForce RTX 3060


In [15]:
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]

In [16]:
#Needed ChatGPT to help with this part, this is why I am submitting this assignment late
# Special tokens for the start and end of sequences
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token

# Mapping words to unique indices and vice versa for English to French translation
word2index = {"SOS": SOS_token, "EOS": EOS_token}; [word2index.setdefault(word, len(word2index)) for pair in english_to_french for word in pair[0].split() + pair[1].split()]
index2word = {i: word for word, i in word2index.items()}

# Custom Dataset class for English to French sentences
class EngFrDataset(Dataset):
    def __init__(self, dataset, word2index):
        # Initialize the dataset and the word-to-index mapping
        self.dataset = dataset
        self.word_to_index = word2index

    def __len__(self):
        # Return the number of items in the dataset
        return len(self.dataset)

    def __getitem__(self, idx):
        # Retrieve an item and its translation, converting words to their indexed representation
        input_sen, target_sentence = self.dataset[idx]
        input_ind = [self.word_to_index[word] for word in input_sen.split()] + [EOS_token]
        target_ind = [self.word_to_index[word] for word in target_sentence.split()] + [EOS_token]
        return torch.tensor(input_ind, dtype=torch.long), torch.tensor(target_ind, dtype=torch.long)


# Initialize the dataset and DataLoader
dataset = EngFrDataset(english_to_french, word2index)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [17]:
class Transformer(nn.Module):
    def __init__(self, input_size, target_size, hidden_size, num_layers, num_heads, dropout=0.03):
        super(Transformer, self).__init__()
        self.embedding_input = nn.Embedding(input_size, hidden_size)
        self.embedding_target = nn.Embedding(target_size, hidden_size)
        self.transformer = nn.Transformer(d_model= hidden_size, nhead= num_heads, num_encoder_layers= num_layers, num_decoder_layers= num_layers, dim_feedforward= hidden_size * 2, dropout= dropout)
        self.fc = nn.Linear(hidden_size, target_size)

    def forward(self, input, target):
        input_embedded = self.embedding_input(input)
        target_embedded = self.embedding_target(target)

        padded_input_mask = self.generate_padding_mask(input)
        padded_target_mask = self.generate_padding_mask(target)
        target_subsequent_mask = self.generate_subsequent_mask(target)

        input_embedded = input_embedded.permute(1, 0, 2)
        target_embedded = target_embedded.permute(1, 0, 2)

        output = self.transformer(input_embedded, target_embedded, src_key_padding_mask= padded_input_mask,
                                  tgt_key_padding_mask= padded_target_mask, 
                                  memory_key_padding_mask= padded_input_mask, 
                                  tgt_mask= target_subsequent_mask
        )

        output = self.fc(output)
        return output.permute(1, 0, 2)

    def generate_padding_mask(self, sequence):
        mask = (sequence == word2index["EOS"])
        return mask.to(device)

    def generate_subsequent_mask(self, sequence):
        mask = (torch.triu(torch.ones(sequence.size(1), sequence.size(1))) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask.to(device)

In [18]:
num_layers=4
num_heads=8
epochs = 101
criterion = nn.CrossEntropyLoss(ignore_index= word2index["EOS"])
a = 0.0001
hidden_size = 1024

#Input & Output Sizes
input_size = len(word2index)
output_size = len(word2index)


model = Transformer(input_size, output_size, hidden_size, num_layers, num_heads).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr= a)


C:\Users\adams\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [19]:
# Training loop
for epoch in range(epochs):
    model.train()
    
    total_loss = 0
    total_correct = 0
    total_examples = 0

    for input_tensor, target_tensor in dataloader:
        input_tensor = input_tensor.to(device)
        target_tensor = target_tensor.to(device)

        optimizer.zero_grad()

        output = model(input_tensor, target_tensor[:, :-1])  # Exclude EOS token from target
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        target_tensor = target_tensor[:, 1:].contiguous().view(-1)

        loss = criterion(output, target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(output, 1)
        correct = (predicted == target_tensor).sum().item()
        total_correct += correct
        total_examples += target_tensor.size(0)
    
    # Print loss every 10 epochs
    if epoch % 10 == 0:
       print(f'Epoch: {epoch}, Loss: {total_loss / len(dataloader)}, Training Acc: {total_correct / total_examples}')

C:\Users\adams\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
C:\Users\adams\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 0, Loss: 6.083161347872251, Training Acc: 0.0026666666666666666
Epoch: 10, Loss: 4.907470443032005, Training Acc: 0.07466666666666667
Epoch: 20, Loss: 4.143514944361402, Training Acc: 0.176
Epoch: 30, Loss: 3.446105228616046, Training Acc: 0.22666666666666666
Epoch: 40, Loss: 2.8943413236698547, Training Acc: 0.30133333333333334
Epoch: 50, Loss: 2.407310545250967, Training Acc: 0.34933333333333333
Epoch: 60, Loss: 2.0400027021185143, Training Acc: 0.4026666666666667
Epoch: 70, Loss: 1.7650345855138518, Training Acc: 0.47733333333333333
Epoch: 80, Loss: 1.5217231723975826, Training Acc: 0.5493333333333333
Epoch: 90, Loss: 1.309900438921018, Training Acc: 0.6106666666666667
Epoch: 100, Loss: 1.1497643835745848, Training Acc: 0.664


In [20]:
def evaluate_model(model, dataloader, criterion):
    total_loss = 0
    total_correct = 0
    total_examples = 0

    model.eval()

    with torch.no_grad(): 
        for input_tensor, target_tensor in dataloader:
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)

            output = model(input_tensor, target_tensor[:, :-1])  # Exclude EOS token from target
            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            target_tensor = target_tensor[:, 1:].contiguous().view(-1)  # Exclude SOS token from target

            loss = criterion(output, target_tensor)

            total_loss += loss.item()

            _, predicted = torch.max(output, 1)
            correct = (predicted == target_tensor).sum().item()
            total_correct += correct
            total_examples += target_tensor.size(0)  # Use target tensor size for total examples

    avg_loss = total_loss / len(dataloader)
    acc = total_correct / total_examples  # Calculate accuracy

    return avg_loss, acc

In [21]:
eval_loss, eval_acc = evaluate_model(model, dataloader, criterion)
print(f"Evaluation Loss: {eval_loss}, Evaluation Accuracy: {eval_acc}")

Evaluation Loss: 0.8505758441162187, Evaluation Accuracy: 0.6986666666666667
